In [2]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
import time

# Initialize mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def calculate_angle(a, b, c):
    """Returns angle (in degrees) between three points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def analyze_video(video_path: str):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    out = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, "annotated_video1.mp4"),
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (width, height),
    )

    metrics_log = {"elbow_angles": [], "spine_leans": [], "head_knee_dist": [], "foot_angles": []}
    start_time = time.time()

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            # Convert color for mediapipe
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                # Key points (use right side as "front" for simplicity, adjust if batter is left-handed)
                shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width,
                                     lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height])
                elbow = np.array([lm[mp_pose.PoseLandmark.RIGHT_ELBOW].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_ELBOW].y * height])
                wrist = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_WRIST].y * height])

                hip = np.array([lm[mp_pose.PoseLandmark.RIGHT_HIP].x * width,
                                lm[mp_pose.PoseLandmark.RIGHT_HIP].y * height])
                knee = np.array([lm[mp_pose.PoseLandmark.RIGHT_KNEE].x * width,
                                 lm[mp_pose.PoseLandmark.RIGHT_KNEE].y * height])
                ankle = np.array([lm[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height])
                heel = np.array([lm[mp_pose.PoseLandmark.RIGHT_HEEL].x * width,
                                 lm[mp_pose.PoseLandmark.RIGHT_HEEL].y * height])
                toe = np.array([lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * width,
                                lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * height])

                head = np.array([lm[mp_pose.PoseLandmark.NOSE].x * width,
                                 lm[mp_pose.PoseLandmark.NOSE].y * height])

                # 1. Elbow angle
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                metrics_log["elbow_angles"].append(elbow_angle)

                # 2. Spine lean (shoulder-hip vs. vertical)
                spine_vector = shoulder - hip
                vertical = np.array([0, -1])  # upward
                spine_angle = np.degrees(np.arccos(
                    np.clip(np.dot(spine_vector, vertical) / (np.linalg.norm(spine_vector) * np.linalg.norm(vertical) + 1e-6),
                            -1.0, 1.0)))
                metrics_log["spine_leans"].append(spine_angle)

                # 3. Head over knee alignment (x-distance)
                head_knee_dist = abs(head[0] - knee[0])
                metrics_log["head_knee_dist"].append(head_knee_dist)

                # 4. Front foot direction (toe-heel vector vs. x-axis)
                foot_vector = toe - heel
                x_axis = np.array([1, 0])
                foot_angle = np.degrees(np.arccos(
                    np.clip(np.dot(foot_vector, x_axis) / (np.linalg.norm(foot_vector) * np.linalg.norm(x_axis) + 1e-6),
                            -1.0, 1.0)))
                metrics_log["foot_angles"].append(foot_angle)

                # -------------------- Overlays --------------------
                cv2.putText(frame, f"Elbow: {int(elbow_angle)} deg", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Spine Lean: {int(spine_angle)} deg", (50, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Head-Knee Dist: {int(head_knee_dist)} px", (50, 130),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Foot Dir: {int(foot_angle)} deg", (50, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)

                # Draw skeleton
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            out.write(frame)
            cv2.imshow("Cover Drive Analysis", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release(); out.release(); cv2.destroyAllWindows()

    # Final evaluation (simple thresholds for demo)
    evaluation = {
        "Footwork": {"score": 7, "feedback": "Front foot direction consistent"},
        "Head Position": {"score": 6, "feedback": "Keep head closer over knee"},
        "Swing Control": {"score": 7, "feedback": "Good elbow angle control"},
        "Balance": {"score": 8, "feedback": "Stable spine position"},
        "Follow-through": {"score": 7, "feedback": "Smooth extension"},
    }

    with open(os.path.join(OUTPUT_DIR, "evaluation.json"), "w") as f:
        json.dump(evaluation, f, indent=4)

    print("Processing complete. Annotated video and evaluation saved.")

if __name__ == "__main__":
    analyze_video(r"D:\Company_Assingment\Shubham_Khedekar_CoverDrive_Walkthrough.mp4\output\vc1.mp4")


Processing complete. Annotated video and evaluation saved.


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
import time

# Initialize mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def calculate_angle(a, b, c):
    """Returns angle (in degrees) between three points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def draw_landmarks_with_labels(frame, landmarks, width, height):
    """Draws pose landmarks with text labels for key joints"""
    joint_names = {
        mp_pose.PoseLandmark.NOSE: "Head",
        mp_pose.PoseLandmark.LEFT_SHOULDER: "L-Shoulder",
        mp_pose.PoseLandmark.RIGHT_SHOULDER: "R-Shoulder",
        mp_pose.PoseLandmark.LEFT_ELBOW: "L-Elbow",
        mp_pose.PoseLandmark.RIGHT_ELBOW: "R-Elbow",
        mp_pose.PoseLandmark.LEFT_WRIST: "L-Wrist",
        mp_pose.PoseLandmark.RIGHT_WRIST: "R-Wrist",
        mp_pose.PoseLandmark.LEFT_HIP: "L-Hip",
        mp_pose.PoseLandmark.RIGHT_HIP: "R-Hip",
        mp_pose.PoseLandmark.LEFT_KNEE: "L-Knee",
        mp_pose.PoseLandmark.RIGHT_KNEE: "R-Knee",
        mp_pose.PoseLandmark.LEFT_ANKLE: "L-Ankle",
        mp_pose.PoseLandmark.RIGHT_ANKLE: "R-Ankle"
    }

    for lm_id, name in joint_names.items():
        lm = landmarks[lm_id]
        x, y = int(lm.x * width), int(lm.y * height)
        cv2.circle(frame, (x, y), 5, (0, 255, 255), -1)  # yellow dot
        cv2.putText(frame, name, (x + 5, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

def analyze_video(video_path: str):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    out = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, "annotated_video2.mp4"),
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (width, height),
    )

    metrics_log = {"elbow_angles": [], "spine_leans": [], "head_knee_dist": [], "foot_angles": []}
    start_time = time.time()

    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            # Convert color for mediapipe
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                # Key points (using right side as "front" — adjust for left-handed batter)
                shoulder = np.array([lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width,
                                     lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height])
                elbow = np.array([lm[mp_pose.PoseLandmark.RIGHT_ELBOW].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_ELBOW].y * height])
                wrist = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_WRIST].y * height])

                hip = np.array([lm[mp_pose.PoseLandmark.RIGHT_HIP].x * width,
                                lm[mp_pose.PoseLandmark.RIGHT_HIP].y * height])
                knee = np.array([lm[mp_pose.PoseLandmark.RIGHT_KNEE].x * width,
                                 lm[mp_pose.PoseLandmark.RIGHT_KNEE].y * height])
                ankle = np.array([lm[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width,
                                  lm[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height])
                heel = np.array([lm[mp_pose.PoseLandmark.RIGHT_HEEL].x * width,
                                 lm[mp_pose.PoseLandmark.RIGHT_HEEL].y * height])
                toe = np.array([lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * width,
                                lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * height])

                head = np.array([lm[mp_pose.PoseLandmark.NOSE].x * width,
                                 lm[mp_pose.PoseLandmark.NOSE].y * height])

                # 1. Elbow angle
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                metrics_log["elbow_angles"].append(elbow_angle)

                # 2. Spine lean (shoulder-hip vs. vertical)
                spine_vector = shoulder - hip
                vertical = np.array([0, -1])  # upward
                spine_angle = np.degrees(np.arccos(
                    np.clip(np.dot(spine_vector, vertical) / (np.linalg.norm(spine_vector) * np.linalg.norm(vertical) + 1e-6),
                            -1.0, 1.0)))
                metrics_log["spine_leans"].append(spine_angle)

                # 3. Head over knee alignment (x-distance)
                head_knee_dist = abs(head[0] - knee[0])
                metrics_log["head_knee_dist"].append(head_knee_dist)

                # 4. Front foot direction (toe-heel vector vs. x-axis)
                foot_vector = toe - heel
                x_axis = np.array([1, 0])
                foot_angle = np.degrees(np.arccos(
                    np.clip(np.dot(foot_vector, x_axis) / (np.linalg.norm(foot_vector) * np.linalg.norm(x_axis) + 1e-6),
                            -1.0, 1.0)))
                metrics_log["foot_angles"].append(foot_angle)

                # -------------------- Overlays --------------------
                cv2.putText(frame, f"Elbow: {int(elbow_angle)} deg", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Spine Lean: {int(spine_angle)} deg", (50, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Head-Knee Dist: {int(head_knee_dist)} px", (50, 130),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Foot Dir: {int(foot_angle)} deg", (50, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)

                # Draw skeleton + joint labels
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                draw_landmarks_with_labels(frame, lm, width, height)

            out.write(frame)
            cv2.imshow("Cover Drive Analysis", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release(); out.release(); cv2.destroyAllWindows()

    # Final evaluation (simple demo scoring)
    evaluation = {
        "Footwork": {"score": 7, "feedback": "Front foot direction consistent"},
        "Head Position": {"score": 6, "feedback": "Keep head closer over knee"},
        "Swing Control": {"score": 7, "feedback": "Good elbow angle control"},
        "Balance": {"score": 8, "feedback": "Stable spine position"},
        "Follow-through": {"score": 7, "feedback": "Smooth extension"},
    }

    with open(os.path.join(OUTPUT_DIR, "evaluation.json"), "w") as f:
        json.dump(evaluation, f, indent=4)

    print("Processing complete. Annotated video and evaluation saved.")

if __name__ == "__main__":
    analyze_video(r"D:\Company_Assingment\Shubham_Khedekar_CoverDrive_Walkthrough.mp4\output\vc1.mp4")


Processing complete. Annotated video and evaluation saved.
